In [1]:
#from google.colab import drive
#drive.mount('/content/drive')

In [2]:
#import package
import pandas as pd
import tensorflow as tf
import tensorflow_hub as hub
import numpy as np
#call training nlp
train_data = pd.read_csv('F:/WORK/Freelancer/yasmin(on going)/yasmin-chatbot/train.txt', sep = ';')#kene tukar utk set path tgk dekat note
train_data.head(10)

,text,emotion
0,i didnt feel humiliated,sadness
1,i can go from feeling so hopeless to so damned...,sadness
2,im grabbing a minute to post i feel greedy wrong,anger
3,i am ever feeling nostalgic about the fireplac...,love
4,i am feeling grouchy,anger
5,ive been feeling a little burdened lately wasn...,sadness
6,ive been taking or milligrams or times recomme...,surprise
7,i feel as confused about life as a teenager or...,fear
8,i have been with petronas for years i feel tha...,joy
9,i feel romantic too,love


In [3]:
#preprocess the data
def preprocess(file):
    path = 'F:/WORK/Freelancer/yasmin(on going)/yasmin-chatbot/'+file+'.txt'#kene tukar set path for each answer
    data = pd.read_csv(path, sep = ';')#separation between the label and the answer
    hos = []
    for i in range(len(data.emotion)):
        if data['emotion'][i] in ['joy', 'love', 'surprise']:#klu data tu label sebagai joy, love ngn  surprise makna nya positive output selain tu negative output
            hos.append(1) # happy is 1
        else:
            hos.append(0) # sad is 0
    data['hos'] = hos
    return data

In [4]:
train_data = preprocess('train')#use the preprocess function create above
train = train_data.copy()
print(train)

                                                    text  emotion  hos
0                                i didnt feel humiliated  sadness    0
1      i can go from feeling so hopeless to so damned...  sadness    0
2       im grabbing a minute to post i feel greedy wrong    anger    0
3      i am ever feeling nostalgic about the fireplac...     love    1
4                                   i am feeling grouchy    anger    0
...                                                  ...      ...  ...
21943  i just had a very brief time in the beanbag an...  sadness    0
21944  i am now turning and i feel pathetic that i am...  sadness    0
21945                     i feel strong and good overall      joy    1
21946  i feel like this was such a rude comment and i...    anger    0
21947  i know a lot but i feel so stupid because i ca...  sadness    0

[21948 rows x 3 columns]


In [5]:
train['emotion'].value_counts()
train['hos'].value_counts()
print(train)

                                                    text  emotion  hos
0                                i didnt feel humiliated  sadness    0
1      i can go from feeling so hopeless to so damned...  sadness    0
2       im grabbing a minute to post i feel greedy wrong    anger    0
3      i am ever feeling nostalgic about the fireplac...     love    1
4                                   i am feeling grouchy    anger    0
...                                                  ...      ...  ...
21943  i just had a very brief time in the beanbag an...  sadness    0
21944  i am now turning and i feel pathetic that i am...  sadness    0
21945                     i feel strong and good overall      joy    1
21946  i feel like this was such a rude comment and i...    anger    0
21947  i know a lot but i feel so stupid because i ca...  sadness    0

[21948 rows x 3 columns]


In [6]:
#method yg kita guna adalah rnn keras package
model = "https://tfhub.dev/google/tf2-preview/gnews-swivel-20dim/1"#pretrained model yg kita akan guna rnn
hub_layer = hub.KerasLayer(model, output_shape=[20], input_shape=[], 
                           dtype=tf.string, trainable=True)
#hub_layer(train['text'][:3])

In [7]:
#how to set the layer
model = tf.keras.Sequential() 
model.add(hub_layer)
model.add(tf.keras.layers.Dense(16, activation='relu'))
model.add(tf.keras.layers.Dense(1))

#model.summary() #boleh uncomment utk tgk summary layer

In [8]:
#parameter yg akan kita guna
model.compile(optimizer='adam',
              loss=tf.losses.BinaryCrossentropy(from_logits=True),
              metrics=[tf.metrics.BinaryAccuracy(threshold=0.0, name='accuracy')])

In [9]:
#preprocess data utk validation pulak
val = preprocess('val')
val['emotion'].value_counts()
val['hos'].value_counts()
#val.head()

0    1080
1     920
Name: hos, dtype: int64

In [10]:
#fit the training process with validation process
history = model.fit(train.text,
                    train.hos,
                    epochs=40,
                    batch_size=512,
                    validation_data=(val.text, val.hos),
                    verbose = 0)

In [11]:
#test the validation result
test = preprocess('val')
results = model.evaluate(test['text'], test['hos'])

print(results)#kat result ni ada 2 output yg depat tuvalidation loss yg blkg tu validation accuracy

63/63 [==============================] - 0s 2ms/step - loss: 0.3895 - accuracy: 0.8660
[0.3895150125026703, 0.8659999966621399]


In [12]:
pred=model.predict(test.text)

preds=[]
for i in range(len(pred)):
	if pred[i]>=0:
		A=1
	else:
		A=0
	preds.append(A)

from sklearn.metrics import classification_report
print(classification_report(preds, test.hos))

              precision    recall  f1-score   support

           0       0.89      0.86      0.88      1114
           1       0.84      0.87      0.85       886

    accuracy                           0.87      2000
   macro avg       0.86      0.87      0.86      2000
weighted avg       0.87      0.87      0.87      2000



In [13]:
#Answers from user will be kept and process in postprocessor
#bot Question
answers1 = []
answers1.append(input('How would you describe your experience at your workplace/college/school in the past few days?'))
answers1.append(input('How do you like to spend your leisure time? How do you feel after it?'))
answers1.append(input('Life has its ups and downs. Although handling successes can be difficult, setbacks can affect mental health strongly. How do you manage your emotions after failures?'))
answers1.append(input('Any recent pleasant experience that you would like to share?'))
answers1.append(input('Any recent unpleasant experience that you would like to share?'))
answers1.append(input('In a broad sense, how would you describe the way your life is going on?'))

#call postprocessor
results = model.predict(answers1)

preds=[]
for i in range(len(results)):
	if results[i]>=0:
		A=1
	else:
		A=0
	preds.append(A)

x = preds.count(1)
y = preds.count(0)

happy = x/len(preds)*100
sad = y/len(preds)*100

print('Score of happy answer:', happy)
print('Score of sad answer:', sad)

Score of happy answer: 0.0
Score of sad answer: 100.0


In [14]:
#Other Question PSS (this will train based on the question strip)
print("Please Choose between 0,1,2,3,4. 0 = never, 1 = almost never, 2 = sometimes, 3 = fairly often, 4 = very often")
answers2 = []
answers2.append(int(input('In the last month, how often have you been upset because of something that happened unexpectedly?')))
answers2.append(int(input('In the last month, how often have you felt that you were unable to control the important things in your life?')))
answers2.append(int(input('In the last month, how often have you felt nervous and stressed?')))
answers2.append(int(input('In the last month, how often have you felt confident about your ability to handle your personal problems?')))
answers2.append(int(input('In the last month, how often have you felt that things were going your way?')))
answers2.append(int(input('In the last month, how often have you found that you could not cope with all the things that you had to do?')))
answers2.append(int(input('In the last month, how often have you been able to control irritations in your life?')))
answers2.append(int(input('In the last month, how often have you felt that you were on top of things?')))
answers2.append(int(input('In the last month, how often have you been angered because of things that happened that were outside of your control?')))
answers2.append(int(input('In the last month, how often have you felt difficulties piling up so high that you could not overcome them?')))

score = []
for i in answers2:
  if i == 0:
    score.append(4)
  elif i == 1:
    score.append(3)
  elif i == 2:
    score.append(2)
  elif i == 3:
    score.append(1)
  elif i == 4:
    score.append(0)
    
accuracy = sum(score)
print('Your stress score is:', accuracy)
#accuracy = (sum(score)/40)*100
#print(accuracy)

Please Choose between 0,1,2,3,4. 0 = never, 1 = almost never, 2 = sometimes, 3 = fairly often, 4 = very often
Your stress score is: 30


In [15]:
#save model
#model.save
# Scores ranging from 0-13 would be considered low stress.
# Scores ranging from 14-26 would be considered moderate stress.
# Scores ranging from 27-40 would be considered high perceived stress.

if accuracy < 13:
  print("U have low stress level")
elif accuracy > 14 & accuracy < 26:
  print("U'r stress level is moderate")
else :
  print("U'r stress level is high")

if sad > happy:
  print("You are going through a bad phase in life. But don't worry, bad times are not permanent.")
else:
  print("Your mental health looks great!!!")

U'r stress level is moderate
You are going through a bad phase in life. But don't worry, bad times are not permanent.


In [18]:
#model.save('F:/WORK/Freelancer/yasmin(on going)/yasmin-chatbot/my_model.h5')
# serialize model to JSON
model_json = model.to_json()
with open("F:/WORK/Freelancer/yasmin(on going)/yasmin-chatbot/model.json", "w") as json_file:
    json_file.write(model_json)

In [16]:
surveys = []
file = open("C:/Users/HTC/Desktop/chatbotv3_2/survey.csv")
for line in file:
    if line == "" or line == "":
        continue
    surveys.append(line.split(","))

score = []
for i in range(9,19):
  if int(surveys[0][i]) == 0:
    score.append(4)
  elif int(surveys[0][i]) == 1:
    score.append(3)
  elif int(surveys[0][i]) == 2:
    score.append(2)
  elif int(surveys[0][i]) == 3:
    score.append(1)
  elif int(surveys[0][i]) == 4:
    score.append(0)
  else:
    score.append(0)

stresslevel = sum(score)
   
if stresslevel < 13:
    res="You have low stress level, Score below 13"
elif stresslevel > 14 & stresslevel < 26:
    res="Your stress level is moderate, Score between 14 and 26"
else :
    res="Your stress level is high. Score above 26."

print(res)


Your stress level is moderate, Score between 14 and 26
